In [1]:
import numpy as np
import torch as th
# import torch.nn as nn
# import torch.nn.functional as F

## Generate separable states

1. Generate random 4 $\times$ 4 matrix
2. Generate a self-adjoint matrix by multiplying the previous matrix by its conjugate transpose
3. Generate a density matrix by dividing the previous matrix by its trace

In [2]:
def generate_random_square_matrices(rows, n_qubits):

    matrices = []
    for _ in range(n_qubits):
        real_part = np.random.rand(rows, rows)
        imag_part = np.random.rand(rows, rows)
        matrices.append(real_part + 1j * imag_part)
    return np.array(matrices)

In [3]:
n_rows = 4

n_qubits = 2

matrices = generate_random_square_matrices(n_rows, n_qubits)
matrices.shape

(2, 4, 4)

In [4]:
def generate_hermitian_product_states(dimensions):
    
    product_states = []
    for dim in dimensions:
        real_part = np.random.rand(dim[0], dim[1])
        imag_part = np.random.rand(dim[0], dim[1])
        product_state = real_part + 1j * imag_part
        product_state = np.matmul(product_state, product_state.conj().T)
        product_state /= np.trace(product_state)
        product_states.append(product_state)
        
    return np.array(product_states)

def generate_orthonormal_matrices(n_rows, n_matrices):
    
    ort_matrices = []
    for _ in range(n_matrices):
        random_matrix = np.random.rand(n_rows, n_rows)
        ort_matrix, _ = np.linalg.qr(random_matrix)
        
        for j in range(n_rows):
            ort_matrix[:, j] /= np.linalg.norm(ort_matrix[:, j])
            
    ort_matrices.append(ort_matrix)
    
    return np.array(ort_matrices)

def generate_orthonormal_product_states(n_rows, n_matrices):
    
    ort_matrices = []
    for _ in range(n_matrices):
        random_matrix = np.random.rand(n_rows, n_rows)
        ort_matrix, _ = np.linalg.qr(random_matrix)
        
        herm = ort_matrix - (1 / n_rows) * np.eye(n_rows)
            
        ort_matrices.append(herm)
    
    return np.array(ort_matrices)
            

In [5]:
ort_matrices = generate_orthonormal_matrices(n_rows=n_rows, n_matrices=n_qubits)

np.trace(ort_matrices[0])

-0.5162622614126825

## Generate the Schmidt coefficients $\alpha$

Generate $N$ random numbers $\alpha_i$ such that
1. $\sum \alpha_i = 1$
2. $\alpha_i > 0$

Do this by:
1. Generate $N$ random numbers between 0 and 1
2. Divide each one of them by their sum

In [9]:
def generate_coefficients(n):
    rand_numbers = np.random.rand(n)
    
    return rand_numbers / np.sum(rand_numbers)

In [16]:
coeffs = generate_coefficients(n_rows)

coeffs, np.sum(coeffs)

(array([0.49377816, 0.03084844, 0.05725369, 0.4181197 ]), 1.0)

## Calculate separable state

Create the separable state by performing

$$\rho_{SEP} = \sum_{i = 1} ^ N \alpha_i \rho_A^i \otimes \rho_B^i$$

In [8]:
def calculate_tensor_product(herm_matrices, coeffs):

    sep_state = np.zeros(herm_matrices.shape[0], dtype=np.complex_128)
    
    for i in range(len(coeffs)):
        # temp_product = np.tensordot(herm_matrices[0, :, i], herm_matrices[1, :, i], axes=0)
        temp_product = np.kron(herm_matrices[0, :, i], herm_matrices[1, :, i])
        
        if(herm_matrices.shape[0] > 2):
            for j in np.arange(2, herm_matrices.shape[0]):
                # temp_product = np.tensordot(temp_product, herm_matrices[j, :, i], axes=0)
                temp_product = np.kron(temp_product, herm_matrices[j, :, i])
                
    tensor_dot_product.append(coeffs[i] * temp_product)
    
    # return tensor_dot_product
    return np.add.reduce(tensor_dot_product)

tensor_dot_product = calculate_tensor_product(herm_matrices, coeffs)

tensor_dot_product.shape

NameError: name 'herm_matrices' is not defined

## Generate many separable states

Unite what we did previously in a single function

In [ ]:
def generate_separable_states(dimensions, n_states):
    sep_states = []
    for _ in range(n_states):
        herm_matrices = generate_hermitian_product_states(dimensions)
        coeffs = generate_coefficients(n_rows)
        tensor_dot_product = calculate_tensor_product(herm_matrices, coeffs)
        sep_states.append(tensor_dot_product)
    return np.array(sep_states)
    

sep_states = generate_separable_states(n_rows, n_qubits, 100)

In [ ]:
sep_states.shape

(100, 4)

In [ ]:
import generate_separable_states as gss

sep_states = gss.generate_separable_states(n_rows, n_qubits, 100)

sep_states.shape

(100, 2, 2)